# Use Keras and hyperparameter optimization (HPO) to recognize hand-written digits with `ibm-watson-machine-learning`

This notebook contains steps and code to demonstrate support of Deep Learning experiments in the Watson Machine Learning service. It introduces commands for data retrieval, training definition persistance, experiment training.

Some familiarity with Python is helpful. This notebook uses Python 3.6.


## Learning goals

The learning goals of this notebook are:

-  Working with the Watson Machine Learning service.
-  Training Deep Learning models (TensorFlow).

## Contents

This notebook contains the following parts:

1.	[Setup](#setup)
2.	[Create model definition](#model_df)
3.	[Train model](#training)

<a id="setup"></a>
## 1. Set up the environment

Before you use the sample code in this notebook, you must perform the following setup tasks:

-  Create a <a href="https://console.ng.bluemix.net/catalog/services/ibm-watson-machine-learning/" target="_blank" rel="noopener no referrer">Watson Machine Learning (WML) Service</a> instance (a free plan is offered and information about how to create the instance can be found <a href="https://dataplatform.ibm.com/docs/content/analyze-data/wml-setup.html" target="_blank" rel="noopener no referrer">here</a>).

### Connection to WML

Authenticate the Watson Machine Learning service on IBM Cloud. You need to provide platform `api_key` and instance `location`.

You can use [IBM Cloud CLI](https://cloud.ibm.com/docs/cli/index.html) to retrieve platform API Key and instance location.

API Key can be generated in the following way:
```
ibmcloud login
ibmcloud iam api-key-create API_KEY_NAME
```

In result, get the value of `api_key` from the output.


Location of your WML instance can be retrieved in the following way:
```
ibmcloud login --apikey API_KEY -a https://cloud.ibm.com
ibmcloud resource service-instance WML_INSTANCE_NAME
```

In result, get the value of `location` from the output.

**Tip**: Your `Cloud API key` can be generated by going to the [**Users** section of the Cloud console](https://cloud.ibm.com/iam#/users). From that page, click your name, scroll down to the **API Keys** section, and click **Create an IBM Cloud API key**. Give your key a name and click **Create**, then copy the created key and paste it below. You can also get a service specific url by going to the [**Endpoint URLs** section of the Watson Machine Learning docs](https://cloud.ibm.com/apidocs/machine-learning).  You can check your instance location in your  <a href="https://console.ng.bluemix.net/catalog/services/ibm-watson-machine-learning/" target="_blank" rel="noopener no referrer">Watson Machine Learning (WML) Service</a> instance details.

You can also get service specific apikey by going to the [**Service IDs** section of the Cloud Console](https://cloud.ibm.com/iam/serviceids).  From that page, click **Create**, then copy the created key and paste it below.

**Action**: Enter your `api_key` and `location` in the following cell.

In [ ]:
api_key = 'PASTE YOUR PLATFORM API KEY HERE'
location = 'PASTE YOUR INSTANCE LOCATION HERE'

In [ ]:
wml_credentials = {
    "apikey": api_key,
    "url": 'https://' + location + '.ml.cloud.ibm.com'
}

In [ ]:
!pip install ibm-watson-machine-learning

### Install and import the `ibm-watson-machine-learning` package
**Note:** `ibm-watson-machine-learning` documentation can be found <a href="http://ibm-wml-api-pyclient.mybluemix.net/" target="_blank" rel="noopener no referrer">here</a>.

In [ ]:
from ibm_watson_machine_learning import APIClient

client = APIClient(wml_credentials)

### Working with spaces

First of all, you need to create a space that will be used for your work. If you do not have space already created, you can use [Deployment Spaces Dashboard](https://dataplatform.cloud.ibm.com/ml-runtime/spaces?context=cpdaas) to create one.


- Click New Deployment Space
- Create an empty space
- Select Cloud Object Storage
- Select Watson Machine Learning instance and press Create

![create space](https://github.com/IBM/skillsnetwork/blob/master/images/create-space.png?raw=true)

- Copy `space_id` and paste it below

![space id](https://github.com/IBM/skillsnetwork/blob/master/images/space-id.png?raw=true)

**Tip**: You can also use SDK to prepare the space for your work. More information can be found [here](https://github.com/IBM/watson-machine-learning-samples/blob/master/cloud/notebooks/python_sdk/instance-management/Space%20management.ipynb).

**Action**: Assign space ID below

In [ ]:
space_id = 'PASTE YOUR SPACE ID HERE'

You can use `list` method to print all existing spaces.

In [ ]:
client.spaces.list(limit=10)

To be able to interact with all resources available in Watson Machine Learning, you need to set **space** which you will be using.



In [ ]:
client.set.default_space(space_id)

### 1.1 Working with Cloud Object Storage

-  Create a [Cloud Object Storage (COS)](https://console.bluemix.net/catalog/infrastructure/cloud-object-storage) instance (a lite plan is offered and information about how to order storage is [here](https://console.bluemix.net/docs/services/cloud-object-storage/basics/order-storage.html#order-storage)).
    - After you create COS instance, go to your COS dashboard.
    - In **Service credentials** tab, click **New Credential**.
    - Add the inline configuration parameter: {["HMAC"](https://console.bluemix.net/docs/services/cloud-object-storage/hmac/credentials.html#using-hmac-credentials):true}, click **Add**.

    This configuration parameter adds the following section to the instance credentials, (for use later in this notebook):
    ```
      "cos_hmac_keys": {
            "access_key_id": "***",
            "secret_access_key": "***"
       }
    ```

    ![cos-credentials](https://github.com/IBM/skillsnetwork/blob/master/images/create_cos_credential.png?raw=true)

The `ibm_boto3` library allows Python developers to manage Cloud Object Storage.

**Note:** If `ibm_boto3` is not preinstalled in you environment please install it by running the following command: `!pip install ibm-cos-sdk`

In [11]:
import ibm_boto3
from ibm_botocore.client import Config
import os
import json
import warnings
warnings.filterwarnings('ignore')

**Action**: Enter your COS credentials in the following cell.  
You can find these credentials in your COS instance dashboard under the **Service credentials** tab.  


 
 ![Image othe HMAC key, described in [set up the environment](#setup) is included in these credentials.

 ![copy-cos-credentials](https://github.com/IBM/skillsnetwork/blob/master/images/cos_credentials.png?raw=true)

In [12]:
cos_credentials = {
  "apikey": "***",
  "cos_hmac_keys": {
    "access_key_id": "***",
    "secret_access_key": "***"
  },
  "endpoints": "***",
  "iam_apikey_description": "***",
  "iam_apikey_name": "***",
  "iam_role_crn": "***",
  "iam_serviceid_crn": "***",
  "resource_instance_id": "***"
}

api_key = cos_credentials['apikey']
service_instance_id = cos_credentials['resource_instance_id']
auth_endpoint = 'https://iam.cloud.ibm.com/identity/token'
service_endpoint = 'https://s3.us-south.cloud-object-storage.appdomain.cloud'

Create the Boto resource by providing type, endpoint_url and credentials.

In [13]:
cos = ibm_boto3.resource('s3',
                         ibm_api_key_id=api_key,
                         ibm_service_instance_id=service_instance_id,
                         ibm_auth_endpoint=auth_endpoint,
                         config=Config(signature_version='oauth'),
                         endpoint_url=service_endpoint)

Create the buckets that you will use to store training data and training results.

**Note:**: Bucket name has to be unique - please update following ones to any unique name.

In [ ]:
buckets = ['tf-keras-data-example', 'tf-keras-results-example']
for bucket in buckets:
    if not cos.Bucket(bucket) in cos.buckets.all():
        print('Creating bucket "{}"...'.format(bucket))
        try:
            cos.create_bucket(Bucket=bucket)
        except ibm_boto3.exceptions.ibm_botocore.client.ClientError as e:
            print('Error: {}.'.format(e.response['Error']['Message']))

The buckets are created.

![buckets-created](https://github.com/IBM/skillsnetwork/blob/master/images/buckets_created.png?raw=true)

In [ ]:
print(list(cos.buckets.limit(50)))

### 1.2 Download the MNIST data and upload it to the COS bucket

In this notebook we work with the Keras **MNIST** sample dataset. Download the training data and upload them to 'mnist-keras-data' bucket.

Following cell creates the 'MNIST_KERAS_DATA' folder and downloads the file from link.

**Note:** First install `wget` library by the following command
`!pip install wget`

In [16]:
link = 'https://s3.amazonaws.com/img-datasets/mnist.npz'

In [19]:
import wget

data_dir = 'MNIST_KERAS_DATA'
if not os.path.isdir(data_dir):
    os.mkdir(data_dir)

if not os.path.isfile(os.path.join(data_dir, os.path.join(link.split('/')[-1]))):
    wget.download(link, out=data_dir)  
        
!ls MNIST_KERAS_DATA

mnist.npz


Upload the data files to created bucket.

In [20]:
bucket_name = buckets[0]
bucket_obj = cos.Bucket(bucket_name)

for filename in os.listdir(data_dir):
    with open(os.path.join(data_dir, filename), 'rb') as data: 
        bucket_obj.upload_file(os.path.join(data_dir, filename), filename)
        print('{} is uploaded.'.format(filename))

mnist.npz is uploaded.


You can see the list of all buckets and their contents.

In [21]:
for obj in bucket_obj.objects.all():
    print('Object key: {}'.format(obj.key))
    print('Object size (kb): {}'.format(obj.size/1024))

Object key: mnist.npz
Object size (kb): 11221.126953125


The model is ready to be trained.

<a id="model_def"></a>
# 2. Create model definition

For the purpose of this example two Keras model definitions have been prepared:

 - Multilayer Perceptron (MLP)
 - Convolution Neural Network (CNN)

### 2.1 Prepare model definition metadata

In [22]:
metaprops = {
    client.model_definitions.ConfigurationMetaNames.NAME: "MNIST mlp model definition",
    client.model_definitions.ConfigurationMetaNames.DESCRIPTION: "MNIST mlp model definition",
    client.model_definitions.ConfigurationMetaNames.COMMAND: "python3 mnist_mlp.py --trainImagesFile ${DATA_DIR}/train-images-idx3-ubyte.gz --trainLabelsFile ${DATA_DIR}/train-labels-idx1-ubyte.gz --testImagesFile ${DATA_DIR}/t10k-images-idx3-ubyte.gz --testLabelsFile ${DATA_DIR}/t10k-labels-idx1-ubyte.gz --learningRate 0.001 --trainingIters 6000",
    client.model_definitions.ConfigurationMetaNames.PLATFORM: {"name": "python", "versions": ["3.7"]},
    client.model_definitions.ConfigurationMetaNames.VERSION: "2.0",
    client.model_definitions.ConfigurationMetaNames.SPACE_UID: space_id
}

### 2.2 Get sample model definition content files from git (Python scripts with CNN and MLP)

In [23]:
filename_mnist = 'MNIST.zip'

if not os.path.isfile(filename_mnist):
    filename_mnist = wget.download('https://github.com/IBM/watson-machine-learning-samples/raw/master/cloud/definitions/keras/mnist/MNIST.zip')

**Tip**: Convert below cell to code and run it to see model deinition's code.

### 2.3 Publish model definition

In [24]:
model_definition_details = client.model_definitions.store(filename_mnist, meta_props=metaprops)

In [25]:
model_definition_id = client.model_definitions.get_id(model_definition_details)
print(model_definition_id)

c94075fe-d4c6-49f8-b161-220e5d67bde9


#### List model definitions

In [26]:
client.model_definitions.list(limit=5)

--------------------------  --------------------  ------------------------------------
NAME                        ASSET_TYPE            GUID
MNIST mlp model definition  wml_model_definition  c94075fe-d4c6-49f8-b161-220e5d67bde9
--------------------------  --------------------  ------------------------------------


<a id="training"></a>
# 3. Train model

### 3.1 Prepare training metadata

In [37]:
training_metadata = {
    client.training.ConfigurationMetaNames.NAME: "Keras-MNIST",
    client.training.ConfigurationMetaNames.SPACE_UID: space_id,
    client.training.ConfigurationMetaNames.DESCRIPTION: "Keras-MNIST predict written digits",
    client.training.ConfigurationMetaNames.TAGS :[{
      "value": "MNIST",
      "description": "predict written difits"
    }],
    client.training.ConfigurationMetaNames.TRAINING_RESULTS_REFERENCE:  {
    "name": "MNIST results",
    "connection": {
            "endpoint_url": service_endpoint,
            "access_key_id": cos_credentials['cos_hmac_keys']['access_key_id'],
            "secret_access_key": cos_credentials['cos_hmac_keys']['secret_access_key']
      },
      "location": {
        "bucket": buckets[0]
      },
    "type": "s3"
  },
  client.training.ConfigurationMetaNames.MODEL_DEFINITION:{
        "id": model_definition_id,
        "command": "python3 mnist_mlp.py --trainImagesFile ${DATA_DIR}/train-images-idx3-ubyte.gz --trainLabelsFile ${DATA_DIR}/train-labels-idx1-ubyte.gz --testImagesFile ${DATA_DIR}/t10k-images-idx3-ubyte.gz --testLabelsFile ${DATA_DIR}/t10k-labels-idx1-ubyte.gz --learningRate 0.001 --trainingIters 6000",
        "hardware_spec": {
          "name": "K80",
          "nodes": 1
        },
        "software_spec": {
          "name": "tensorflow_2.1-py3.7"
        },
        "parameters": {
          "name": "MNIST mlp",
          "description": "Simple MNIST mlp model"
        }
      },
  client.training.ConfigurationMetaNames.TRAINING_DATA_REFERENCES: [
       {
      "name": "training_input_data",
      "type": "s3",
      "connection": {
        "endpoint_url": service_endpoint,
        "access_key_id": cos_credentials['cos_hmac_keys']['access_key_id'],
        "secret_access_key": cos_credentials['cos_hmac_keys']['secret_access_key']
      },
      "location": {
        "bucket": buckets[1]
      },
      "schema": {
        "id":"idmlp_schema",
        "fields": [
          {
            "name": "text",
            "type": "string"
          }
        ]
      }
    }
  ]
}

### 3.2 Train model in background

In [38]:
training = client.training.run(training_metadata)

### 3.3 Get training id and status

In [39]:
training_id = client.training.get_uid(training)


Please regularly execute the cell below to monitor your job. Once "status" changed from "pending" or "running" to "completed", we can check on the results in Cloud Object Store.

In [43]:
client.training.get_status(training_id)["state"]

'completed'

### 3.4 Get training details

In [ ]:
training_details = client.training.get_details(training_id)
print(json.dumps(training_details, indent=2))

#### List trainings

In [ ]:
client.training.list(limit=5)

Please open your bucket again in the IBM Cloud Object Store admin interface. Please find the "training-log.txt" file.

![download log](https://github.com/IBM/skillsnetwork/blob/master/images/download-training-log.png?raw=true)

Please select it by ticking the check box left to it and then click on "Download objects". Please submit this file to the grading system for peer assesment.

This concludes the assignment.

### Author

**Jan Sołtysik**, Intern in Watson Machine Learning.

Updated for **Using GPUs to Scale and Speed-up Deep Learning edX **  by  <a href="https://www.linkedin.com/in/samaya-madhavan"> Samaya Madhavan </a>




Copyright © 2020 IBM. This notebook and its source code are released under the terms of the MIT License.